# Satelite Data analysis with the use of machine learning algorithms for land use estimation

######### First the needed libraries need to be entered  ( if any need to be installed please do so) ############################

In [ ]:
import numpy as np
from skimage import io
from skimage import exposure
from skimage.color import rgb2gray
import shutil
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import cm
from matplotlib import colors
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.patches as mpatches
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from PIL import Image
import glob
import os
import rasterio
from osgeo import gdal
import PIL
from PIL import TiffImagePlugin
from shapely.geometry import Point, Polygon, box
import torch
import fiona
import rasterio
import rasterio.mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas as gpd
from rasterio import plot
from rasterio.mask import mask
from rasterio.plot import show_hist
import geopandas as gpd
from rasterio.plot import show
from rasterio.plot import plotting_extent
import zipfile
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import HTML
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pathlib import Path
import netCDF4 as nc
import rasterio
from shapely.geometry import box
from fiona.crs import from_epsg
from osgeo import gdal, gdalconst
import pycrs
from rasterio.crs import CRS
import rioxarray as rxr
import earthpy as et
import xarray as xr 
import tifftools
from tifffile import imread, imwrite

# The paths should be entered on the cell bellow #

In [ ]:
data_path='name of path'
data_path_full='name of satellite image path'

In [ ]:
os.mkdir(data_path_full+'/resampled')
resampled=data_path_full+'/resampled/'

######### Finding band files #########


In [ ]:

bandPath = data_path_full
bandNames = os.listdir(bandPath)
bandNames.remove('resampled')
bandNames

In [ ]:
lst = [os.path.splitext(x)[0] for x in bandNames]
lst

# Please insert the path to the shapefile bellow #

In [ ]:
aoiFile = fiona.open('path to shapefile')
aoiGeom = [aoiFile[0]['geometry']]
print(aoiFile.crs)

########## Cropping of alla band with the shapefile #########

In [ ]:
import rasterio as rio
for band in lst:
    with rio.open(bandPath+band+'.jp2') as src:
        out_image, out_transform=mask(src,aoiGeom,crop=True)
        out_meta=src.meta.copy()
        out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform}) 
    with rio.open(os.path.join(bandPath+band+'.tif'),'w',**out_meta) as dest:
        dest.write(out_image)
               

######### Resampling all bands to 10m resolution #########

In [ ]:
from osgeo import gdal, gdalconst
bandname='T34TFL_20220408T092029_'
bandlist=['B01','B05','B06','B07','B11','B12','B8A','B09','B10']
referenceFile = "B04.tif"
for ban in bandlist:
    #inputfile = 'B09.tif'
    input2 = gdal.Open(bandPath+bandname+ban+'.tif', gdalconst.GA_ReadOnly)
    inputProj = input2.GetProjection()
    inputTrans = input2.GetGeoTransform()

#referencefile = #Path to reference file
    reference = gdal.Open(bandPath+bandname+referenceFile, gdalconst.GA_ReadOnly)
    referenceProj = reference.GetProjection()
    referenceTrans = reference.GetGeoTransform()
    bandreference = reference.GetRasterBand(1)    
    x = reference.RasterXSize 
    y = reference.RasterYSize


    #outputfile = 'B01t.tif'
    driver= gdal.GetDriverByName('GTiff')
    output = driver.Create(bandPath+bandname+ban+'s.tif',x,y,1,bandreference.DataType)
    output.SetGeoTransform(referenceTrans)
    output.SetProjection(referenceProj)

    gdal.ReprojectImage(input2,output,inputProj,referenceProj,gdalconst.GRA_Bilinear)

    del output
    del reference


In [ ]:

usedband=['B01s','B02','B03','B04','B05s','B06s','B07s','B08','B8As','B09s','B10s','B11s','B12s']
for name in usedband:
    shutil.move(data_path_full+bandname+name+'.tif', resampled+bandname+name+'.tif')
 

############ Creating indices ############

In [ ]:
import rasterio as rio
bandname='T34TFL_20220408T092029_'
bandlist=['B01','B05','B06','B07','B11','B12','B8A','B09','B10']
b2=rio.open(resampled+bandname+'B02.tif')
b3=rio.open(resampled+bandname+'B03.tif')
b4=rio.open(resampled+bandname+'B04.tif')
b8=rio.open(resampled+bandname+'B08.tif')
b9=rio.open(resampled+bandname+'B09s.tif')
b11=rio.open(resampled+bandname+'B11s.tif')
b12=rio.open(resampled+bandname+'B12s.tif')



In [ ]:
B2=b2.read()
B3=b3.read()
B4=b4.read()
B8=b8.read()
B9=b9.read()
B11=b11.read()
B12=b12.read()
# Normalized Difference Vegetation index
ndvi = (B8.astype(float)-B4.astype(float))/(B8+B4)
undvi=np.nan_to_num(ndvi)
meta = b4.meta 
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)
with rio.open(os.path.join(resampled, 'undvi.tif'), 'w', **meta) as ds1t:
    ds1t.write(undvi.astype(rio.float32))
#Normalized Difference Water Index    
ndwi=(B3.astype(float)-B8.astype(float))/(B3+B8)
undwi=np.nan_to_num(ndwi)
meta = b4.meta 
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)
with rio.open(os.path.join(resampled, 'undwi.tif'), 'w', **meta) as dst2:
    dst2.write(undwi.astype(rio.float32))
# Normalized Difference Built-up Index    
ndbi=(B11.astype(float)-B8.astype(float))/(B8+B11)
undbi=np.nan_to_num(ndbi)
meta = b4.meta 
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)
with rio.open(os.path.join(resampled, 'undbi.tif'), 'w', **meta) as dst3:
    dst3.write(undbi.astype(rio.float32)) 
#Built-up Index    
bui=ndbi-ndvi  
bui[np.isinf(bui)]=10
ubui=np.nan_to_num(bui)
meta = b4.meta 
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)
with rio.open(os.path.join(resampled, 'ubui.tif'), 'w', **meta) as dst4:
    dst4.write(ubui.astype(rio.float32))    
# Bare Soil Index    
bsi=((B11+B4)-(B8+B2))/((B11+B4)+(B8+B2))
bsi[np.isinf(bsi)]=10
ubsi=np.nan_to_num(bsi)
meta = b4.meta 
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)
with rio.open(os.path.join(resampled, 'ubsi.tif'), 'w', **meta) as dst4:
    dst4.write(ubsi.astype(rio.float32)) 
# Green Chlorophyl  Index
gci=(B9/B3)-1
gci[np.isinf(gci)]=10
ugci=np.nan_to_num(gci)
meta = b4.meta 
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)
with rio.open(os.path.join(resampled, 'ugci.tif'), 'w', **meta) as dst4:
    dst4.write(ugci.astype(rio.float32))
# Moisture Stress Index
msi=(B11/B8)
msi[np.isinf(msi)]=10
umsi=np.nan_to_num(msi)
meta = b4.meta 
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)
with rio.open(os.path.join(resampled, 'umsi.tif'), 'w', **meta) as dst4:
    dst4.write(umsi.astype(rio.float32))

######### Visualization of target area #######

In [ ]:
def composition(band1,band2,band3):
    im_comp=np.dstack([band1/band1.max(), band2/band2.max(), band3/band3.max()])
    for i in range(3):
        v_min, v_max = np.percentile(im_comp[:,:,i],(1,98))
        im_comp[:,:,i] = exposure.rescale_intensity(im_comp[:,:,i],in_range=(v_min,v_max))
    return im_comp

In [ ]:


coll = io.ImageCollection(resampled + '*.tif')

# a list of loaded files
print(coll.files)

band_names=['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12','BSI','BUI','GCI','MSI','NDBI','NDVI','NDWI']

In [ ]:
im_comp = composition(coll[3],coll[2],coll[1])
fig=plt.figure(figsize=(20,10))
plt.imshow(im_comp)

# Please insert training files path bellow #

In [ ]:
trainpath='insert train path here'
p_water = np.loadtxt(trainpath+'water.txt',dtype='int')
p_shallow_water=np.loadtxt(trainpath+'shallow_water.txt',dtype='int')
p_concrete = np.loadtxt(trainpath+'concrete.txt',dtype='int')
p_brick=np.loadtxt(trainpath+'brick.txt', dtype='int')
p_dry_soil_crop=np.loadtxt(trainpath+'dry_soil_and_dry_crop.txt', dtype='int')
p_wet_soil=np.loadtxt(trainpath+'wet_ soil.txt', dtype='int')
p_low_veg = np.loadtxt(trainpath+'low_vegetation.txt',dtype='int')
p_trees = np.loadtxt(trainpath+'tree_cover.txt',dtype='int')

LC_class_names = ['water', 'shallow water','concrete','brick','wet soil','dry soil and crop','low veg','tree cover']
#LC_class_names = ['water', 'artificial', 'low vegetation', 'tree cover']

######## visualization of training points #######

In [ ]:
# marking the classes examples
im_comp_p=np.zeros(im_comp.shape)
im_comp_p[:,:,0]=rgb2gray(im_comp)
im_comp_p[:,:,1]=rgb2gray(im_comp)
im_comp_p[:,:,2]=rgb2gray(im_comp)

# marking in blue the training points for 'water' class
im_comp_p[p_water[:,1],p_water[:,0],0] =0
im_comp_p[p_water[:,1],p_water[:,0],1] =0
im_comp_p[p_water[:,1],p_water[:,0],2] =1

# marking in bright blue the training points for 'shallow water' class
im_comp_p[p_shallow_water[:,1],p_shallow_water[:,0],0] =0
im_comp_p[p_shallow_water[:,1],p_shallow_water[:,0],1] =1
im_comp_p[p_shallow_water[:,1],p_shallow_water[:,0],2] =1

# marking in white the training points for 'concrete' class
im_comp_p[p_concrete[:,1],p_concrete[:,0],0] =1
im_comp_p[p_concrete[:,1],p_concrete[:,0],1] =1
im_comp_p[p_concrete[:,1],p_concrete[:,0],2] =1

# marking in red the training points for 'brick' class
im_comp_p[p_brick[:,1],p_brick[:,0],0] =1
im_comp_p[p_brick[:,1],p_brick[:,0],1] =0
im_comp_p[p_brick[:,1],p_brick[:,0],2] =0

# marking in purple the training points for 'dry soil and crop' class
im_comp_p[p_dry_soil_crop[:,1],p_dry_soil_crop[:,0],0] =0.255
im_comp_p[p_dry_soil_crop[:,1],p_dry_soil_crop[:,0],1] =0.211
im_comp_p[p_dry_soil_crop[:,1],p_dry_soil_crop[:,0],2] =0.67

# marking in pink the training points for 'wet soil' class
im_comp_p[p_wet_soil[:,1],p_wet_soil[:,0],0] =0.82
im_comp_p[p_wet_soil[:,1],p_wet_soil[:,0],1] =0.3
im_comp_p[p_wet_soil[:,1],p_wet_soil[:,0],2] =0.7

# marking in yellow the training points for 'low vegetation' class
im_comp_p[p_low_veg[:,1],p_low_veg[:,0],0] =1
im_comp_p[p_low_veg[:,1],p_low_veg[:,0],1] =1
im_comp_p[p_low_veg[:,1],p_low_veg[:,0],2] =0

# marking in green the training points for 'tree cover' class
im_comp_p[p_trees[:,1],p_trees[:,0],0] =0
im_comp_p[p_trees[:,1],p_trees[:,0],1] =1
im_comp_p[p_trees[:,1],p_trees[:,0],2] =0



plt.figure(figsize=(20,10))
plt.imshow(im_comp_p)
 

########### Stacking the classes examples #########

In [ ]:
bands=[]
for i in range(len(coll)):
    band = np.asarray(coll[i].data)[p_water[:,1],p_water[:,0]]
    bands.append(band)
X_water=np.stack(bands).T
print(X_water.shape)

bands=[]
for i in range(len(coll)):
    band = np.asarray(coll[i].data)[p_shallow_water[:,1],p_shallow_water[:,0]]
    bands.append(band)
X_shallow_water=np.stack(bands).T
print(X_shallow_water.shape)

bands=[]
for i in range(len(coll)):
    band = np.asarray(coll[i].data)[p_concrete[:,1],p_concrete[:,0]]
    bands.append(band)
X_concrete=np.stack(bands).T
print(X_concrete.shape)

bands=[]
for i in range(len(coll)):
    band = np.asarray(coll[i].data)[p_brick[:,1],p_brick[:,0]]
    bands.append(band)
X_brick=np.stack(bands).T
print(X_brick.shape)

bands=[]
for i in range(len(coll)):
    band = np.asarray(coll[i].data)[p_dry_soil_crop[:,1],p_dry_soil_crop[:,0]]
    bands.append(band)
X_dry_soil_crop=np.stack(bands).T
print(X_dry_soil_crop.shape)

bands=[]
for i in range(len(coll)):
    band = np.asarray(coll[i].data)[p_wet_soil[:,1],p_wet_soil[:,0]]
    bands.append(band)
X_wet_soil=np.stack(bands).T
print(X_wet_soil.shape)

bands=[]
for i in range(len(coll)):
    band = np.asarray(coll[i].data)[p_low_veg[:,1],p_low_veg[:,0]]
    bands.append(band)
X_low_veg=np.stack(bands).T
print(X_low_veg.shape)

bands=[]
for i in range(len(coll)):
    band = np.asarray(coll[i].data)[p_trees[:,1],p_trees[:,0]]
    bands.append(band)
X_trees=np.stack(bands).T
print(X_trees.shape)


# stacking data for all classes in one table
X=np.vstack((X_water,X_shallow_water,X_concrete,X_brick,X_dry_soil_crop,X_wet_soil,X_low_veg,X_trees))
# add X_brick after X_concrete for all the learning stacks
print(X.shape)
y=np.vstack((np.ones((X_water.shape[0], 1)),
             2*np.ones((X_shallow_water.shape[0], 1)),
             3*np.ones((X_concrete.shape[0], 1)),
             4*np.ones((X_brick.shape[0], 1)),
             5*np.ones((X_dry_soil_crop.shape[0], 1)),
             6*np.ones((X_wet_soil.shape[0], 1)),
             7*np.ones((X_low_veg.shape[0], 1)),
             8*np.ones((X_trees.shape[0], 1))))

print(y.shape)

# K-fold Cross validation #

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# https://scikit-learn.org/0.15/modules/generated/sklearn.cross_validation.KFold.html
true = []; pred = []

kfold = KFold(n_splits=10, random_state=42, shuffle=True)
for train_index, test_index in kfold.split(X):
    model = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=42)
    
    x_train = X[train_index]; x_test = X[test_index]
    y_train = y[train_index]; y_test = y[test_index]
    
    model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    
    true.append(y_test)
    pred.append(predictions)
    
cv_results = cross_val_score(model, X, y, cv=kfold, scoring='accuracy', verbose=10)

print(cv_results.mean(), cv_results.std())
cf=confusion_matrix(np.concatenate(true), np.concatenate(pred)

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    #plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
labels=['water','shallow water','urban','brick','dry soil & veg','wet soil','low veg','trees']
plot_confusion_matrix(cf,labels,title='Confusion matrix',cmap='YlGnBu', normalize=False)

# Random forest creation and data classification

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100,
                             max_depth=5,
                             random_state=42)    

rf_clf.fit(X, np.ravel(y))

In [ ]:
N=10

for i in range(N):
    fig = plt.figure(figsize=(40,20))
    plot_tree(rf_clf.estimators_[i],
              feature_names = band_names, 
              class_names = LC_class_names ) 
    plt.show()

In [ ]:
bands=[]
for i in range(len(coll)):
    band = np.asarray(coll[i].data).flatten()
    bands.append(band)

X_all=np.stack(bands).T
X_all.shape


In [ ]:
%%time
Y_pred_rf = rf_clf.predict(X_all)


In [ ]:
Y_im_rf = Y_pred_rf.reshape(coll[1].shape)
Y_im_rf.shape


In [ ]:

Y_im_rf

# Result visualization

In [ ]:
ccm= ListedColormap(["midnightblue","teal","dimgrey","black","khaki","saddlebrown","lime","darkgreen"]) # custom colormap
water=mpatches.Patch(color='midnightblue',label='water')
shallowwater=mpatches.Patch(color='teal',label='shallow water')
concrete=mpatches.Patch(color='dimgrey',label='urban')
brick=mpatches.Patch(color='black',label='brick')
drs=mpatches.Patch(color='khaki',label='dry soil and vegetation')
ws=mpatches.Patch(color='saddlebrown',label='wet soil')
lv=mpatches.Patch(color='lime',label='low vegetation')
tr=mpatches.Patch(color='darkgreen',label='trees')


In [ ]:
from PIL import Image
import rasterio as rio
im=Image.fromarray(Y_im_rf)
im.save('ts2.tif')
b4=rio.open(reproject+'B4.tif')
i=rio.open('ts2.tif')
B4=b4.read()
Im=i.read()
meta = b4.meta 
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)
with rio.open(os.path.join(data_path_full, 'Full Classification.tif'), 'w', **meta) as dst:
    dst.write(Im.astype(rio.float32))

In [ ]:
#img=rio.open(os.path.join(data_path_full, 'tci.tif'))
cls=rio.open(os.path.join(data_path_full, 'Full Classification.tif'))
c=cls.read()
plt.figure(1,figsize=(20,10))
plt.title('Satellite Image')
plt.ylabel('Northing (m)')
plt.xlabel('Easting (m)')
plt.imshow(im_comp)

fig=plt.figure(2,figsize=(20,10))
plt.legend(handles=[water,shallowwater,concrete,brick,drs,ws,lv,tr],bbox_to_anchor=(1.04,1), loc="upper left")
plt.title('Classification')
plt.ylabel('Northing (m)')
plt.xlabel('Easting (m)')
#plt.savefig("ts.jpeg",dpi=600)
show(cls,cmap=ccm,interpolation='none')
fig.savefig(data_path_full+"Full Classification.jpeg",dpi=600)
plt.close()
